# Scam Detector: Decision Tree

    This Jupyter Notebook will be used to run a Decision Tree Algorithm to predict if a given email is a scam or a ham(a normal email).

## Import Packages

In [60]:
#import the packages we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

## Retrieve Data

In [61]:
col_names = ['sender', 'receiver', 'subject', 'body', 'label', 'urls']
path = "./data/CEAS_08.csv"
data1 = pd.read_csv(path)
data1 = data1.drop('date', axis=1)
col_names[-1], col_names[-2] = col_names[-2], col_names[-1]
data1 = data1[col_names]
data1.head(10)

,sender,receiver,subject,body,urls,label
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,1,0
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1
5,Caroline Aragon <dwthaidomainnamesm@thaidomain...,user7-ext5@gvc.ceas-challenge.cc,From Caroline Aragon,\n\n\n\n\nYo wu urS mo ou go rc ebo eForM rgi ...,0,1
6,Replica Watches <jhorton@thebakercompanies.com>,user2.10@gvc.ceas-challenge.cc,Replica Watches,We have fake Swiss Men's and Ladie's Replica \...,0,1
7,Daily Top 10 <acidirev_1972@tcwpg.com>,user2.3@gvc.ceas-challenge.cc,CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
8,qydlqcws-iacfym@issues.apache.org,xrh@spamassassin.apache.org,[Bug 5780] URI processing turns uuencoded stri...,http://issues.apache.org/SpamAssassin/show_bug...,1,0
9,Daily Top 10 <orn|dent_1973@musicaedischi.it>,user7@gvc.ceas-challenge.cc,CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1


In [62]:
col_names = ['sender', 'receiver', 'subject', 'body', 'label', 'urls']
path = "./data/Nazario.csv"
data2 = pd.read_csv(path)
data2 = data2.drop('date', axis=1)
col_names[-1], col_names[-2] = col_names[-2], col_names[-1]
data2 = data2[col_names]
print(data2.shape)
data2.head(10)

(1565, 6)


,sender,receiver,subject,body,urls,label
0,Mail System Internal Data <MAILER-DAEMON@monke...,NaN,DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL DATA,This text is part of the internal format of yo...,1,1
1,cPanel <service@cpanel.com>,jose@monkey.org,Verify Your Account,Business with \t\t\t\t\t\t\t\tcPanel & WHM \t...,1,1
2,Microsoft Outlook <recepcao@unimedceara.com.br>,NaN,Helpdesk Mailbox Alert!!!,Your two incoming mails were placed on pending...,1,1
3,Ann Garcia <AnGarcia@mcoe.org>,"""info@maaaaa.org"" <info@maaaaa.org>",IT-Service Help Desk,Password will expire in 3 days. Click Here To ...,0,1
4,"""USAA"" <usaaacctupdate@sccu4u.com>",Recipients <usaaacctupdate@sccu4u.com>,Final USAA Reminder - Update Your Account Now,"To ensure delivery to your inbox, please add U...",1,1
5,=?utf-8?Q?P=2eSupport?= <infos@2015p.com>,jose@monkey.org,=?utf-8?Q?Dear=20Client=20=3a=20Update=20Your=...,"PayPal Secure Dear Client, ...",1,1
6,Update,Recipients,Update,08/10/2011 06:14:36 am +0800 - en_US.UTF-8 - O...,1,1
7,"""USAA"" <usaa-offers@banking2.org>",Recipients <usaa-offers@banking2.org>,IMPORTANT NEW MESSAGE FROM USAA,E-mail Address Update\t View Accounts | Pri...,1,1
8,"""Tourne, Michele"" <MTourne@co.humboldt.ca.us>",Undisclosed recipients:;,Mailbox Warning!!!,"Dear Valued User, We are having a majo...",1,1
9,"""English, Saundra (FDNY)"" <Saundra.English@fdn...","""info@eeee.org"" <info@eeee.org>",Help Desk Password Update,Password will expire in 3 days. Click Here To ...,1,1


In [63]:
col_names = ['sender', 'receiver', 'subject', 'body', 'label', 'urls']
path = "./data/Nigerian_Fraud.csv"
data3 = pd.read_csv(path)
data3 = data3.drop('date', axis=1)
col_names[-1], col_names[-2] = col_names[-2], col_names[-1]
data3 = data3[col_names]
print(data3.shape)
data3.head(10)

(3332, 6)


,sender,receiver,subject,body,urls,label
0,MR. JAMES NGOLA. <james_ngola2002@maktoob.com>,webmaster@aclweb.org,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...,0,1
1,Mr. Ben Suleman <bensul2004nng@spinfinder.com>,R@M,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",0,1
2,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1
3,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1
4,Maryam Abacha <m_abacha03@www.com>,R@M,I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope...",0,1
5,Kuta David <davidkuta@postmark.net>,davidkuta@yahoo.com,Partnership,ATTENTION: ...,0,1
6,Barrister tunde dosumu <tunde_dosumu@lycos.com>,NaN,Urgent Attention,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)...",1,1
7,William Drallo <william2244drallo@maktoob.com>,webmaster@aclweb.org,URGENT BUSINESS PRPOSAL,FROM: WILLIAM DRALLO.\nCONFIDENTIAL TEL: 233-2...,0,1
8,MR USMAN ABDUL <abdul_817@rediffmail.com>,R@M,THANK YOU,"CHALLENGE SECURITIES LTD.\nLAGOS, NIGERIA\n\n\...",0,1
9,Tunde Dosumu <barrister_td@lycos.com>,NaN,Urgent Assistance,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)...",1,1


In [64]:
col_names = ['sender', 'receiver', 'subject', 'body', 'label', 'urls']
path = "./data/SpamAssasin.csv"
data4 = pd.read_csv(path)
data4 = data4.drop('date', axis=1)
col_names[-1], col_names[-2] = col_names[-2], col_names[-1]
data4 = data4[col_names]
print(data4.shape)
data4.head(10)

(5809, 6)


,sender,receiver,subject,body,urls,label
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",1,0
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",1,0
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,1,0
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,1,0
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",1,0
5,"""Martin Adamson"" <martin@srv0.ems.ed.ac.uk>",zzzzteana@yahoogroups.com,Re: [zzzzteana] Nothing like mama used to make,> I just had to jump in here as Carbonara is o...,1,0
6,"""Martin Adamson"" <martin@srv0.ems.ed.ac.uk>",zzzzteana@yahoogroups.com,[zzzzteana] Playboy wants to go out with a bang,The Scotsman - 22 August 2002\n\n Playboy want...,1,0
7,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,Re: [zzzzteana] Nothing like mama used to make,Martin Adamson wrote: > > Isn't it just basic...,1,0
8,"""Martin Adamson"" <martin@srv0.ems.ed.ac.uk>",zzzzteana@yahoogroups.com,[zzzzteana] Meaningful sentences,The Scotsman\n\n Thu 22 Aug 2002 \n\n Meaningf...,1,0
9,"""NOI Administrator"" <admin@networksonline.com>",<spamassassin-talk@example.sourceforge.net>,[SAtalk] SA CGI Configurator Scripts,I have been trying to research via SA mirrors ...,1,0


In [65]:
data = pd.concat([data1, data2, data3, data4])
data.shape

(49860, 6)

In [66]:
send_mode = data['sender'].mode()[0]
data['sender'].fillna(send_mode, inplace=True)
sub_mode = data['subject'].mode()[0]
data['subject'].fillna(sub_mode, inplace=True)
body_mode = data['body'].mode()[0]
data['body'].fillna(body_mode, inplace=True)

data = data.drop('receiver', axis=1)
data['sender'].isnull().any()

False

## TF-IDF

In [69]:
corpus_sender = data['sender'][0:]
send_lengths = np.array([len(text.split()) for text in corpus_sender])
vectorizer_send = TfidfVectorizer()
send = vectorizer_send.fit_transform(corpus_sender)
print(vectorizer_send.get_feature_names_out().shape)
print(send.shape)

(60164,)
(49860, 60164)


In [70]:
send_sum = np.array(send.sum(axis=1)).flatten()
send_normalized = send_sum / (send_lengths + 1e-6)
send_normalized.shape

(49860,)

In [71]:
"""
corpus_rec = data['receiver'][0:]
rec_lengths = np.array([len(text.split()) for text in corpus_rec])
vectorizer_rec = TfidfVectorizer()
rec = vectorizer_rec.fit_transform(corpus_rec)
print(vectorizer_rec.get_feature_names_out().shape)
print(rec.shape)
"""

"\ncorpus_rec = data['receiver'][0:]\nrec_lengths = np.array([len(text.split()) for text in corpus_rec])\nvectorizer_rec = TfidfVectorizer()\nrec = vectorizer_rec.fit_transform(corpus_rec)\nprint(vectorizer_rec.get_feature_names_out().shape)\nprint(rec.shape)\n"

In [72]:
"""
rec_sum = np.array(rec.sum(axis=1)).flatten()
rec_normalized = rec_sum / (rec_lengths + 1e-6)
rec_normalized.shape
"""

'\nrec_sum = np.array(rec.sum(axis=1)).flatten()\nrec_normalized = rec_sum / (rec_lengths + 1e-6)\nrec_normalized.shape\n'

In [73]:
corpus_sub = data['subject'][0:]
sub_lengths = np.array([len(text.split()) for text in corpus_sub])
vectorizer_sub = TfidfVectorizer()
sub = vectorizer_sub.fit_transform(corpus_sub)
print(vectorizer_sub.get_feature_names_out().shape)
print(sub.shape)

(20339,)
(49860, 20339)


In [74]:
sub_sum = np.array(sub.sum(axis=1)).flatten()
sub_normalized = sub_sum / (sub_lengths + 1e-6)
sub_normalized.shape

(49860,)

In [75]:
corpus_body = data['body'][0:]
body_lengths = np.array([len(text.split()) for text in corpus_body])
vectorizer_body = TfidfVectorizer()
body = vectorizer_body.fit_transform(corpus_body)
print(vectorizer_body.get_feature_names_out().shape)
print(body.shape)

(430730,)
(49860, 430730)


In [76]:
body_sum = np.array(body.sum(axis=1)).flatten()
body_normalized = body_sum / (body_lengths + 1e-6)
body_normalized.shape

(49860,)

In [77]:
data['body']= body_normalized
data['subject']= sub_normalized
#data['receiver']= rec_normalized
data['sender']= send_normalized

In [78]:
data.head(10)

,sender,subject,body,urls,label
0,0.607310,0.353270,0.124521,1,1
1,1.111647,0.576933,0.316441,1,1
2,0.646334,0.558736,0.026796,1,1
3,0.677584,0.343630,0.001701,1,0
4,0.690363,0.999999,1.438519,1,1
5,0.689102,0.539430,0.169125,0,1
6,0.687392,0.706895,0.199024,0,1
7,0.515883,0.558736,0.026827,1,1
8,2.200211,0.245090,0.156281,1,0
9,0.595439,0.558736,0.026819,1,1


## Decision Class

In [79]:
class Decision:
    """ A decision is used to ask the question at a decision node to split the data.
    This class records column number and values and matches the stored feature value to a give feature value
    """
    
    def __init__(self, feature_index, threshold):
        self.feature_index = feature_index
        self.threshold = threshold
        
    def ask(self, input):
        # Compares input feature value to stored value
        feature_val = input[self.feature_index]
        if isinstance(feature_val, (int, float, np.number)):
            return feature_val >= self.threshold
        else:
            return feature_val == self.threshold
        

## Helper Functions for Splitting

In [80]:
def divide_df(rows, decision):
    # Partitions a data frame
    # Check if each row matches decision, divide into true and false
    col = rows[:, decision.feature_index]
    if np.issubdtype(col.dtype, np.number):
        mask = col >= decision.threshold
    else:
        mask = col == decision.threshold
    left, right = rows[mask],rows[~mask]
    return left, right

In [81]:
def label_count(rows):
    # Counts the number of each classification in data frame
    y = rows[:, -1]
    unique, label_counts = np.unique(y, return_counts=True)
    return dict(zip(unique,label_counts))

In [82]:
def gini_impurity(rows):
    #Calculates Gini Impurity for a data frame of rows.
    y = rows[:, -1]
    _, label_counts = np.unique(y, return_counts=True)
    probs = label_counts/label_counts.sum()
    return 1.0 - np.sum(probs**2)

In [83]:
def info_gain(left, right, curr_gini):
    #Information gain: Gini of the root node subtracted by the impurty of the two children nodes.
    if len(left) + len(right) == 0:
        return 0
    prob = float(len(left) / (len(left) + len(right)))
    return curr_gini - prob * gini_impurity(left) - (1 - prob) * gini_impurity(right)
                 

In [84]:
def threshold_candidates(col, max_thresh=5):
    #Choose candidate threshold split
    unique = np.unique(col)
    if len(unique) > max_thresh:
        quantile = np.linspace(0, 100, max_thresh + 2)[1:-1]
        unique = np.percentile(unique, quantile)
    if len(unique) > 1:
        return (unique[:-1] + unique[1:])/2
    else:
        return unique

In [85]:
def info_gain_split(rows):
    #Find best decision to make based on informaiton gain
    X = rows[:, :-1]
    y = rows[:, -1]
    curr_gini = gini_impurity(rows)
    feature_count = X.shape[1]
    
    highest_gain = 0
    optimal_decision = None
    
    for feature_index in range(feature_count):
        col = X[:, feature_index]
        
        #Candidate Thresholds
        thresholds = threshold_candidates(col) if np.issubdtype(col.dtype, np.number) else np.unique(col)
        
        for candidate in thresholds:
            if np.issubdtype(col.dtype, np.number):
                mask = col >= candidate
            else:
                mask = col == candidate
            
            if mask.sum() == 0 or mask.sum() == len(mask):
                continue
        
            left, right = rows[mask], rows[~mask]
            gain = info_gain(left, right, curr_gini)
            
            if gain > highest_gain:
                highest_gain, optimal_decision = gain, Decision(feature_index, candidate)
                
    return highest_gain, optimal_decision

## Build Tree and Node Classes

In [86]:
class LeafNode:
    # A leaf Node holdes classified data.
    # Holds a dictionary with class counts in the leaf.
    
    def __init__(self,rows):
        self.pred = label_count(rows)

In [87]:
class DecisionNode:
    # A Decision Node asks a Decision to be made.
    # Holds reference to a Decision, and two child nodes.
    
    def __init__(self, decision, left, right):
        self.decision = decision
        self.left = left
        self.right = right

In [128]:
def build_tree(rows, depth=0, max_depth=10, min_sample_split=2):
    # Recursively Builds tree.
    if len(rows) < min_sample_split or depth >= max_depth:
        return LeafNode(rows)
    
    highest_gain, optimal_decision = info_gain_split(rows)
    
    #Base case no further gain
    if highest_gain == 0 or optimal_decision is None:
        return LeafNode(rows)
    
    #Found Partition
    left, right = divide_df(rows, optimal_decision)
    
    #Recurse Left Subtree
    left_subtree = build_tree(left, depth+1, max_depth, min_sample_split)
    
    #Recurse Right Subtree
    right_subtree = build_tree(right, depth+1, max_depth, min_sample_split)
    
    #Return Decision Node
    return DecisionNode(optimal_decision, left_subtree, right_subtree)

In [114]:
#print(data.shape)
#x = data.iloc[:, :-1]
#x
my_tree = build_tree(data.to_numpy()[0:40000])

In [115]:
def predict(row, curr_node):
    #Base Case: Curr node is a leaf
    if isinstance(curr_node, LeafNode):
        total = sum(curr_node.pred.values())
        return max(curr_node.pred, key=curr_node.pred.get), {k: v/total for k,v in curr_node.pred.items()}
    
    #Recurse the left or right subtree
    if curr_node.decision.ask(row):
        return predict(row, curr_node.left)
    else:
        return predict(row, curr_node.right)

## Prediction and Testing

In [150]:
test_data = {'sender': ['luna_prado@gmail.com'], 'subject': ['Advisor Help'], 'body':['Hello Dr. Athienitis, can you help me with choosing classes for the upcoming semester. Look forward to staying in contact.'], 'urls':[0]}
test_df = pd.DataFrame(test_data)

In [151]:
#test_data = {'sender': ['asjfnakjsnfkanf@gmail.com'], 'subject': ['SCAM ALERT'], 'body':['Make money quick, urgent new opportunity. Please buy now for your future. Passive Income, Easy life.'], 'urls':[1]}
#test_df = pd.DataFrame(test_data)

In [152]:
new_body = vectorizer_body.transform(test_df['body'])
new_body.shape

(1, 430730)

In [153]:
new_body = vectorizer_body.transform(test_df['body'])
new_body_sum = new_body.sum(axis=1).flatten()
nonzero_body = (new_body > 0).sum(axis=1).flatten()

new_body_normalized = (new_body_sum/nonzero_body if nonzero_body > 0 else 0)

new_send = vectorizer_send.transform(test_df['sender'])
new_send_sum = new_send.sum(axis=1).flatten()
nonzero_send = (new_send > 0).sum(axis=1).flatten()

new_send_normalized = (new_send_sum/nonzero_send if nonzero_send > 0 else 0)

"""
new_rec = vectorizer_rec.transform(test_df['receiver'])
new_rec_sum = new_rec.sum(axis=1).flatten()
nonzero_rec = (new_rec > 0).sum(axis=1).flatten()

new_rec_normalized = (new_rec_sum/nonzero_rec if nonzero_rec > 0 else 0)
"""

new_sub = vectorizer_sub.transform(test_df['subject'])
new_sub_sum = new_sub.sum(axis=1).flatten()
nonzero_sub = (new_sub > 0).sum(axis=1).flatten()

new_sub_normalized = (new_sub_sum/nonzero_sub if nonzero_sub > 0 else 0)

test_df['body'] = new_body_normalized
test_df['sender'] = new_send_normalized
#test_df['receiver'] = new_rec_normalized
test_df['subject'] = new_sub_normalized
test_df

,sender,subject,body,urls
0,0.656144,1.0,0.198892,0


In [154]:
predict(test_df.to_numpy()[0], my_tree)

(1.0, {1.0: 1.0})

In [113]:
#test_df.to_numpy()[0]

In [123]:
print(data.to_numpy()[40002])
predict(data.to_numpy()[40002], my_tree)

[0.79483138 0.42632895 0.02673031 1.         1.        ]


(0.0, {0.0: 1.0})

## Random Forest Draft:

In [148]:
class RandomForest:
    def __init__(self, tree_count=10, max_depth=10, min_sample_split=2, feature_count=None):
        self.tree_count = tree_count
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.feature_count = feature_count
        self.trees = []
        
    def fit(self, X, y):
        self.trees = []
        
        for _ in range(self.tree_count):
            X_partial, y_partial = self.bootstrap(X, y)
            rows = np.concatenate((X_partial, y_partial), axis=1)
            
            tree = build_tree(rows, max_depth=self.max_depth, min_sample_split=self.min_sample_split)
            self.trees.append(tree)
        
        
    def bootstrap(self, X, y):
        sample_count = X.shape[0]
        row_index = np.random.choice(sample_count, sample_count, replace=True)
        return X[row_index], y[row_index]
        
        
    def random_predict(self, X):
        preds = np.array([predict(X, tree) for tree in self.trees])
        final_preds = np.swapaxes(preds, 0, 1)
        #Need to implement mode getter
        return final_preds

In [130]:
rows1 = np.array([[0, 2, 3, 4], [1, 3, 5, 6]])
ys = np.array([[0],[1]])
print(rows1)
print(ys)
print(np.concatenate((rows1, ys), axis=1))

[[0 2 3 4]
 [1 3 5 6]]
[[0]
 [1]]
[[0 2 3 4 0]
 [1 3 5 6 1]]


In [149]:
X = data.to_numpy()[:,:-1]
y = data.to_numpy()[:, -1].reshape(-1,1)
my_forest = RandomForest()
my_forest.fit(X,y)

In [161]:
my_forest.random_predict(test_df.to_numpy()[0])[0]

array([1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0], dtype=object)